In [5]:
from frame_classification_process import frame_process
from keras.utils import to_categorical
import numpy as np
import os

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
#### grab small part of training data ####
num_videos = 50

curr_path = os.getcwd()
x_train_ind = np.load(curr_path+'/datasets/x_train_ind_above400.npy')
y_train = np.load(curr_path+'/datasets/y_train_mapped_above400.npy')
 
if num_videos > len(y_train) or None: 
    num_videos = len(y_train)
    
x_train_ind = x_train_ind[:num_videos]
y_train_ori = y_train[:num_videos]
unique, counts = np.unique(y_train_ori, return_counts=True)
num_classes = len(unique)

fp = frame_process(num_videos, size = (64,64,3))
X_train = fp.process_updates_frameSeq_stacked(x_train_ind)
y_train = to_categorical(y_train_ori, num_classes=num_classes)


100%|██████████| 50/50 [00:01<00:00, 33.44it/s]


In [4]:
#### load full training data ####
curr_path = os.getcwd()
X_train = np.load("./datasets/X_train_10_64_64_3.npy")
y_train = np.load(curr_path+'/datasets/y_train_mapped_above400.npy')
unique, counts = np.unique(y_train, return_counts=True)
num_classes = len(unique)
y_train = to_categorical(y_train, num_classes=num_classes)

In [35]:
#### check data ####
print("X shape: ", X_train.shape, "y shape: ", y_train.shape)
print("frame size: ", X_train.shape[2:])
print("num_videos:", num_videos)
print("num_classes", num_classes)
y_count = dict(zip(unique, counts))
print(y_count)

X shape:  (50, 10, 64, 64, 3) y shape:  (50, 10)
frame size:  (64, 64, 3)
num_videos: 50
num_classes 10
{0: 8, 1: 2, 2: 13, 3: 1, 4: 5, 5: 5, 6: 2, 7: 5, 8: 4, 9: 5}


In [36]:
#### create model ####
from models import models
c3d = models(model_name = "c3d",  lr = 1e-4, reg = 0, size = (64,64,3))
# c3d.load(model_path, model_name, model_name)

(10, 64, 64, 3)
(None, 4, 29, 29, 32)
(None, 2, 12, 12, 64)
(None, 1, 5, 5, 128)
(None, 256)
(None, 256)
(None, 10)


In [ ]:
#### training ####
# c3d = models(model_name = "c3d",  lr = 1e-5, reg = 1e-6, size = (64,64,3))
c3d.train(X_train, y_train, epochs=2000, batch_size=32, validation_split = 0, verbose = 1, )
# c3d.save(model_path , model_name, model_name)

Epoch 1/2000
50/50 [==============================] - 0s - loss: 3.0567 - acc: 0.2200     
Epoch 2/2000
50/50 [==============================] - 0s - loss: 3.6066 - acc: 0.1400     
Epoch 3/2000
50/50 [==============================] - 0s - loss: 2.6961 - acc: 0.2600     
Epoch 4/2000
50/50 [==============================] - 0s - loss: 3.5716 - acc: 0.2000     
Epoch 5/2000
50/50 [==============================] - 0s - loss: 3.1077 - acc: 0.2000     
Epoch 6/2000
50/50 [==============================] - 0s - loss: 3.1868 - acc: 0.1400     
Epoch 7/2000
50/50 [==============================] - 0s - loss: 3.2013 - acc: 0.1400     
Epoch 8/2000
50/50 [==============================] - 0s - loss: 3.1053 - acc: 0.1800     
Epoch 9/2000
50/50 [==============================] - 0s - loss: 3.6996 - acc: 0.2000     
Epoch 10/2000
50/50 [==============================] - 0s - loss: 3.4019 - acc: 0.1400     
Epoch 11/2000
50/50 [==============================] - 0s - loss: 3.4062 - acc: 0.1600   

In [46]:
#### result ####
ypred = c3d.model.predict(X_train)
ypred_label = np.argmax(ypred, axis = 1)
print('Test Accuracy: {0}'.format(np.mean(ypred_label == y_train_ori)))
# c3d.plot()


Test Accuracy: 0.26


In [49]:
print(ypred_label)
print(ypred)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2]
[[ 0.06030669  0.04851756  0.39228705  0.02318109  0.04803038  0.12900102
   0.05179254  0.07145704  0.12239737  0.05302931]
 [ 0.06030669  0.04851756  0.39228705  0.02318109  0.04803038  0.12900102
   0.05179254  0.07145704  0.12239737  0.05302931]
 [ 0.06030669  0.04851756  0.39228705  0.02318109  0.04803038  0.12900102
   0.05179254  0.07145704  0.12239737  0.05302931]
 [ 0.06030669  0.04851756  0.39228705  0.02318109  0.04803038  0.12900102
   0.05179254  0.07145704  0.12239737  0.05302931]
 [ 0.06030669  0.04851756  0.39228705  0.02318109  0.04803038  0.12900102
   0.05179254  0.07145704  0.12239737  0.05302931]
 [ 0.06030669  0.04851756  0.39228705  0.02318109  0.04803038  0.12900102
   0.05179254  0.07145704  0.12239737  0.05302931]
 [ 0.06030669  0.04851756  0.39228705  0.02318109  0.04803038  0.12900102
   0.05179254  0.07145704  0.12239737  0.05302931]
 [ 0.06030669  0.04851